In [2]:
import utils
from dataset import BuildingDataset
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from model import get_transform
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import torch

In [11]:
# Example list of tensors of different sizes
tensor_list = [torch.randn(2, 3), torch.randn(4), torch.randn(3, 5, 6)]

# Step 1: Flatten each tensor
flattened_tensors = [t.flatten() for t in tensor_list]

# Step 2: Concatenate the flattened tensors
continuous_tensor = torch.cat(flattened_tensors)

print(continuous_tensor)
print(continuous_tensor.shape)

tensor([ 0.4481, -0.2339,  1.9366, -0.3357, -0.0180,  2.2093, -1.2660, -0.0180,
         0.7344,  1.4809,  1.2207,  0.9514, -1.1552,  0.6162,  0.4376, -1.7551,
         2.1437,  1.1757, -0.4266,  0.6852, -0.3957, -1.4685,  2.3706,  0.1332,
        -0.6268, -1.7921,  1.3558,  1.8049,  1.9941,  0.7323,  1.4645, -1.4719,
         1.0485,  0.0092, -0.6252,  0.2871,  0.1052, -1.0610,  0.1958,  0.4416,
        -0.5750,  1.6747, -0.2971,  0.8896, -1.3586, -0.0590, -0.2260,  1.2747,
        -1.8033,  0.7617, -0.6508,  0.0710, -0.5775, -0.0805, -0.5263,  0.7292,
        -0.2923,  0.9309, -3.1173,  3.0424, -1.4423, -0.0460,  0.2827, -0.4675,
        -0.3647,  0.9484,  0.0208,  0.3188, -0.3123,  0.6580, -0.4157, -2.7303,
        -0.4883, -0.4634, -0.6228,  1.8601,  0.6374,  1.9224,  0.9352, -0.3682,
         0.7273, -1.2279,  0.3356, -0.1458,  0.1916, -0.6775, -1.9377,  0.2285,
         1.4374, -0.3871, -0.6581, -0.3516, -2.1266, -0.6455, -1.2120, -0.4273,
        -1.0665, -0.9022,  0.5503, -0.09

In [ ]:
# Load data for exploration.
dataset_test = BuildingDataset(
    "datasets/mlc_training_data/images_annotated/",
)

In [ ]:
# Load data for exploration.
dataset_expl = BuildingDataset(
    "datasets/mlc_training_data/images_annotated/",
)

data = [dataset_expl[i] for i in range(10)]
# data = [d for d in dataset_expl]

In [ ]:
img, masks, boxes = data[2][0], data[2][1]["masks"], data[2][1]["boxes"]

height_labels = [str(h.item()) for h in data[2][1]["building_heights"]]


def show_segmentation(img, masks, boxes=None, bcolors="red"):
    output_image = draw_segmentation_masks(img, masks.to(torch.bool), alpha=0.8)
    if boxes is not None:
        output_image = draw_bounding_boxes(
            output_image, boxes, height_labels, font_size=15, colors=bcolors
        )

    plt.figure(figsize=(12, 12))
    plt.imshow(output_image.permute(1, 2, 0))


show_segmentation(img, masks, boxes)

In [ ]:
height_labels = [d[1]["building_heights"] for d in data]
height_labels = torch.cat(height_labels).numpy()

counts, bins = np.histogram(
    height_labels,
    bins=[3, 6, 9, 15, 25, 40, 70, 120, 200, max(250, height_labels.max())],
)
plt.stairs(counts, bins, fill=True)

for q in [0.25, 0.5, 0.75, 0.95, 0.99]:
    print(f"Quantile {q}: {np.quantile(height_labels, q)}")

In [ ]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

dataset = BuildingDataset(
    "datasets/mlc_training_data/images_annotated/",
    transforms=get_transform(train=True),
)

data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    num_workers=4,
    collate_fn=utils.collate_fn,
)

# For Training
images, targets = next(iter(data_loader))

images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)
print(predictions[0])

In [ ]:
from model import train

train(
    dataset_root="datasets/mlc_training_data/images_annotated",
    num_epochs=20,
    train_batch_size=10,
)

In [ ]:
from typing import Any


class Engine:
    name = "super shit"

    def __init__(self) -> None:
        self.year = "2020"

    def start(self):
        print("engines starts")

    def stop(self):
        print("engines stops")


class Car:
    def __init__(self, engine) -> None:
        self.engine = engine

    def __getattribute__(self, name: str) -> Any:
        if name == "forward":
            

        return getattr(super().__getattribute__("engine"), name)

    def stop(self):
        print(self.engine.year)
        print(self.engine.name)


engine = Engine()
car = Car(engine)

car.stop()
car.start()